<a href="https://colab.research.google.com/github/BNkosi/Zeus/blob/master/Zeus_starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zeus - a closed domain question answering chatbot

<img src ="https://infobush.com/wp-content/uploads/2020/01/Facts-About-Zeus.jpg" align="left">

**Team Members**: B. Nkosi, K. Galela, K. Mnguni, N. Msibi, N. Magudulela, O. Mkhuhlane, T. Muthego, V. Mthemba

* [Model Repo](https://github.com/BNkosi/Zeus)
* [Data Repo]
* [Model Trello Board](https://trello.com/b/pNaYe3pe/zeus)
* [Data Trello Board]

## Table of Contents
---
1. [Introduction](#intro)
  * Background
  * Problem Statement
---
2. [Imports](#imports)
  * Libraries
  * Data
---
3. [Modelling](#model)
  * Basic QA Pipeline
  * Model selection
  * Retriever selection
  * Fine-tuning
---
4. [Evaluation](#evaluation)
---
5. [Model Analysis](#analysis)
  * Results
---
6. [Conclusion](#conclusion)
---
7. [References](#ref)
---
  


<a id="intro"></a>
## 1. Introduction
### Background

The EDSA QA chatbot has two parts - Onboarding Chatbot and Zues.

### Onboarding Chatbot

Your co-pilot when you join the company or step into a new role

We all have access to the internet and the majority of us have become very good at using search engines to find the kind of information we need. The world is also slowly moving toward interactive search functionality such as chatbots 

Many corporates have built chatbot with Q+A functionality to improve customer experience and rapidly return the kinds of basic answers that customers need to make their product purchasing decisions. However, this technology appears to be vastly underutilized within the internal staff structures of company. Most corporate staff work within a team and division and their intimate knowledge of company policies, procedures, best practices and tools fall within the domain of that team or division. If they need to work on something that is usually foreign to their role, they have to ask a member of another team or division.

This either takes place in the form of an email (cumbersome and often gets ignored) or being pestered in person (distracting and annoying at times). This tool would be trained on the companies internal policies, regulations, best practices, corporate communications, tool documentations etc. to generate an understanding of how to best answer questions that staff may have about things that they are not intimately aware of but need near real-time feedback on to continue with their own projects.

### Zeus

Answer Domain specific questions to aid the user when they need it

Covid-19 brings new challenges to the education sector in that learning can no longer take place face-to-face. A key tool in education is learner engagement in the form of live questions and answers and the instructor being able to gauge whether they’ve lost their audience to re-explain a key concept in different ways. A key skill of an instructor is to be able to support their answers with reference to principles and provide relatable examples.

Company X would like to build a virtual instructor that can answer complex questions accompanying their online media content. This tool would answer a content viewers’ questions even though they are not viewing live and eventually stand alone as a student resource. The tool would be trained on student handbooks, past exams, suggested solutions, video-content, etc. in order to present an answer with extracts from key text and examples to the students’ question.

### Problem Statement

1. Build an closed domain question answering pipeline to answer textual questions.

2. Train the pipeline on:
  * Companies internal policies, regulations, best practices, corporate communications, etc. - Onboarding
  * EDSA problem statement, preprocessing, trains and other external resources.

<a id="imports"></a>
## Imports

### Libraries

In [ ]:
# Required Libraries
!pip install farm-haystack

In [7]:
from haystack import Finder
from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

To continue, please check out the tutorials in the [haystack](https://github.com/deepset-ai/haystack) repository